In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_scheduler

2025-01-25 18:47:27.505384: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-25 18:47:27.543886: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-25 18:47:27.543915: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-25 18:47:27.543937: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-25 18:47:27.551688: I tensorflow/core/platform/cpu_feature_g

In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [4]:
# Loading Dataframe
df = pd.read_excel('df_complete_cleansing.xlsx', index_col=0)
df

,ReceiverID,ActionType,NegoOutcome,Content,Content_Length,Sentence_Count,Word_Count_nltk,NegoOutcomeLabel,Sentiment,SentimentCategory
SenderID,,,,,,,,,,
70,69,Offer,FinalAccept,hope well management company agreed building f...,2529,17,470,1,0.213699,Positive
69,70,Counteroffer,FinalAccept,reaching mei apologize slight delay getting ba...,2579,21,483,1,0.165002,Positive
70,69,Counteroffer,FinalAccept,r kind response need hurry guess face daily fi...,2336,15,454,1,0.222533,Positive
69,70,Counteroffer,FinalAccept,swift reply read set proposal discussed collea...,1961,13,381,1,0.160333,Positive
70,69,Counteroffer,FinalAccept,proposal effort far seems getting close resolv...,1917,10,376,1,0.122500,Positive
...,...,...,...,...,...,...,...,...,...,...
851,856,Counteroffer,FinalReject,latest offer point completely let u take corpo...,1755,21,358,0,0.285340,Positive
856,851,Counteroffer,FinalReject,much latest offer happy could already reached ...,1460,17,294,0,0.282807,Positive
851,856,Counteroffer,FinalReject,meyer fast answer happy found solution issue n...,673,10,144,0,0.251786,Positive


In [5]:
# Cerverting Content to string
df['Content'] = df['Content'].astype(str)

In [6]:
# Extracting the sentences and labels from the dataframe
sentences = df['Content'].tolist()
labels = df['NegoOutcomeLabel'].values


In [8]:
labels

array([1, 1, 1, ..., 0, 0, 0])

In [9]:
# Define the model name to be used
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

# Load the tokenizer from the pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the pre-trained model for sequence classification with 2 labels
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Tokenize the input sentences
tokenized_texts = tokenizer(
    sentences, max_length=512, truncation=True, padding=True, return_tensors="pt"
)

print(tokenized_texts[0])
print(tokenized_texts[1])

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [11]:
# Extract the input IDs from the tokenized textsk
input_ids = tokenized_texts["input_ids"]

# Extract the attention masks from the tokenized texts
attention_masks = tokenized_texts["attention_mask"]

In [14]:
# Split the input IDs and labels into training and validation sets with test size of 30%
train_inputs, validation_inputs, train_labels, validation_labels= train_test_split(input_ids, labels, random_state=42, test_size=0.3)

# Split the attention masks into training and validation sets
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.3)

In [15]:
train_inputs

tensor([[    0, 35685,   162,  ...,     1,     1,     1],
        [    0, 13650,  1294,  ...,     1,     1,     1],
        [    0, 23242,  7153,  ...,     1,     1,     1],
        ...,
        [    0,   879, 45779,  ...,     1,     1,     1],
        [    0,  1638,   857,  ...,     1,     1,     1],
        [    0, 32835,  6296,  ...,     1,     1,     1]])

In [16]:
validation_inputs

tensor([[    0, 35685, 19300,  ...,     1,     1,     1],
        [    0, 35685,  4343,  ...,     1,     1,     1],
        [    0, 35356,  1263,  ...,     1,     1,     1],
        ...,
        [    0, 23156, 10939,  ...,     1,     1,     1],
        [    0,   119,  1397,  ...,     1,     1,     1],
        [    0, 10932,   206,  ...,     1,     1,     1]])

In [17]:
# Convert the input IDs and labels into PyTorch tensors
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

#convert the attention masks into PyTorch tensors
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [18]:
#Batchsize for training
batch_size = 32

# Create a DataLoader for the training data with the specified batch size
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler=RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create a DataLoader for the validation data with the specified batch size
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler=SequentialSampler(validation_data)
validation_dataloader =DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [19]:
# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the selected device (GPU or CPU)
model.to(device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [20]:
# Move the model to the selected device (GPU or CPU)
model.to(device)

# Set the model to training mode
model.train()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [21]:
# Initialize the AdamW optimizer with the model parameters and a learning rate of 5e-5
optimizer = AdamW(model.parameters(), lr=5e-5)

In [22]:
# Define a function to compute the accuracy of the model
def compute_accuracy(model, data_loader, device):

    # Disable gradient calculation for evaluation
    with torch.no_grad():

        # Initialize counters for correct predictions and total examples
        correct_pred, num_examples = 0, 0

        # Iterate over batches in the data loader
        for batch_idx, batch in enumerate(data_loader):
            # Prepare data by moving input IDs, attention masks, and labels to the device (GPU or CPU)
            input_ids, input_mask, labels = [t.to(device) for t in batch]  # Move to GPU

            # Perform a forward pass through the model
            outputs = model(input_ids, attention_mask=input_mask, labels=labels)
            loss, logits = outputs['loss'], outputs['logits']

            # Get the predicted labels by finding the index of the max logit
            _, predicted_labels = torch.max(logits, 1)

            # Update the total number of examples and correct predictions
            num_examples += labels.size(0)
            correct_pred += (predicted_labels == labels).sum()

    # Calculate and return the accuracy as a percentage
    return correct_pred.float()/num_examples * 100

In [23]:
import time

# Record the start time of the training process
start_time = time.time()

# Set the number of training epochs
NUM_EPOCHS = 10

# Loop over each epoch
for epoch in range(NUM_EPOCHS):
    
    # Set the model to training mode
    model.train()
    
    # Loop over each batch in the training data loader
    for batch_idx, batch in enumerate(train_dataloader):
        # Prepare data by moving input IDs, attention masks, and labels to the device (GPU or CPU)
        input_ids, input_mask, labels = [t.to(device) for t in batch]  # Move to GPU
        
        # Perform a forward pass through the model
        outputs = model(input_ids, attention_mask=input_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']
    
        # Perform backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Log the training progress every 250 batches
        if not batch_idx % 250:
            print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_dataloader):04d} | '
                   f'Loss: {loss:.4f}')

    # Set the model to evaluation mode        
    model.eval()

    # Compute and print the training and validation accuracy
    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_dataloader, device):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, validation_dataloader, device):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

Epoch: 0001/0010 | Batch 0000/0051 | Loss: 0.6685
training accuracy: 68.50%
valid accuracy: 69.43%
Time elapsed: 0.54 min
Epoch: 0002/0010 | Batch 0000/0051 | Loss: 0.6863
training accuracy: 68.50%
valid accuracy: 69.43%
Time elapsed: 1.08 min
Epoch: 0003/0010 | Batch 0000/0051 | Loss: 0.6305
training accuracy: 68.75%
valid accuracy: 65.14%
Time elapsed: 1.62 min
Epoch: 0004/0010 | Batch 0000/0051 | Loss: 0.6038
training accuracy: 78.55%
valid accuracy: 72.71%
Time elapsed: 2.16 min
Epoch: 0005/0010 | Batch 0000/0051 | Loss: 0.5002
training accuracy: 84.19%
valid accuracy: 74.00%
Time elapsed: 2.70 min
Epoch: 0006/0010 | Batch 0000/0051 | Loss: 0.3563
training accuracy: 81.50%
valid accuracy: 72.57%
Time elapsed: 3.24 min
Epoch: 0007/0010 | Batch 0000/0051 | Loss: 0.4081
training accuracy: 97.67%
valid accuracy: 74.29%
Time elapsed: 3.78 min
Epoch: 0008/0010 | Batch 0000/0051 | Loss: 0.1108
training accuracy: 98.47%
valid accuracy: 73.14%
Time elapsed: 4.32 min
Epoch: 0009/0010 | Batch

In [24]:
from sklearn.metrics import classification_report
from collections import Counter

# After training, set the model to evaluation mode
model.eval()

# Initialize lists to store all predictions and labels
all_predictions = []
all_labels = []

# Disable gradient calculation for evaluation
with torch.no_grad():
    # Iterate over batches in the validation data loader
    for batch in validation_dataloader:
       # Prepare data by moving input IDs, attention masks, and labels to the device (GPU or CPU)
        input_ids, input_mask, labels = [t.to(device) for t in batch]
        
        # Perform a forward pass through the model
        outputs = model(input_ids, attention_mask=input_mask)
        logits = outputs.logits  
        
        # Collect predictions and labels
        predictions = torch.argmax(logits, dim=1)  
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Count the number of predictions for each class
prediction_counts = Counter(all_predictions)
print("\nPrediction Counts:")
for label, count in prediction_counts.items():
    print(f"Class {label}: {count} predictions")

# Print the classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_predictions, target_names=["Class 0", "Class 1"]))


Prediction Counts:
Class 1: 598 predictions
Class 0: 102 predictions

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.71      0.34      0.46       214
     Class 1       0.76      0.94      0.84       486

    accuracy                           0.75       700
   macro avg       0.73      0.64      0.65       700
weighted avg       0.75      0.75      0.72       700



In [25]:
import pandas as pd
from sklearn.metrics import classification_report
from collections import Counter
import torch

# After training, set the model to evaluation mode
model.eval()

# Initialize lists to store all predictions and labels
all_predictions = []
all_labels = []

# Disable gradient calculation for evaluation
with torch.no_grad():
    # Iterate over batches in the validation data loader
    for batch in validation_dataloader:
        # Prepare data by moving input IDs, attention masks, and labels to the device (GPU or CPU)
        input_ids, input_mask, labels = [t.to(device) for t in batch]

        # Perform a forward pass through the model
        outputs = model(input_ids, attention_mask=input_mask)
        logits = outputs.logits  # Logits contain the raw predictions of the model

        # Collect predictions and labels
        predictions = torch.argmax(logits, dim=1)  # Class with the highest probability
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Count the predictions
prediction_counts = Counter(all_predictions)

# Classification report as dictionary
classification_report_dict = classification_report(
    all_labels, all_predictions, target_names=["Class 0", "Class 1"], output_dict=True
)

# Prepare data for transposed table
result_data = {}

# Add model name as the first column
model_name = "distilroberta_financial-10-epochs"
result_data["Model"] = [model_name]

# Add prediction counts
for label, count in prediction_counts.items():
    result_data[f"Prediction Count - Class {label}"] = [count]

# Add classification report
for metric, values in classification_report_dict.items():
    if isinstance(values, dict):
        for sub_metric, value in values.items():
            result_data[f"{metric} - {sub_metric}"] = [value]
    else:
        result_data[metric] = [values]

# Create DataFrame
result_df = pd.DataFrame(result_data)

# Save as CSV
result_df.to_csv("evaluation_results_distil_roberta_financial_10_epochs.csv", index=False)

print("CSV file has been successfully saved:evaluation_distilroberta_financial_10_epochs.csv")

CSV file has been successfully saved:evaluation_distilroberta_financial_10_epochs.csv
